## Interannotator Agreement

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("tweet_corpus.csv")
# Extract handlabelled tweets
dfmanual = df.iloc[:3000]
print(dfmanual.head())
print(dfmanual.shape)
print("Dataframe imported and loaded")

pd.crosstab(dfmanual['Sentiment Analysis (Label)'], dfmanual['label2'])

                       created_at        id_str  \
0  Mon Feb 01 23:59:59 +0000 2021  1.356392e+18   
1  Mon Feb 01 23:59:58 +0000 2021  1.356392e+18   
2  Mon Feb 01 23:59:52 +0000 2021  1.356392e+18   
3  Mon Feb 01 23:59:38 +0000 2021  1.356392e+18   
4  Mon Feb 01 23:59:33 +0000 2021  1.356392e+18   

                                           full_text  quote_count  \
0  Here is today’s Idaho COVID-19 vaccine data at...          0.0   
1  Health care worker dies after second dose of C...          0.0   
2  Boris Johnson: Emergency Services to receive C...          0.0   
3  Think about it.... The same Liberal government...          0.0   
4  South Africa receives its first anti-Covid vac...          0.0   

   reply_count retweet_count  favorite_count lang             user_id_str  \
0          1.0           0.0             0.0   en            2284308170.0   
1          0.0           0.0             0.0   en              77677956.0   
2          0.0           0.0             0.0   

label2,Negative,Neutral,Positive
Sentiment Analysis (Label),,,
Negative,465,127,71
Neutral,90,1315,149
Positive,15,101,667


In [2]:
# Export eval.csv
eval = dfmanual.iloc[:1000]
eval.drop(['label2'], axis=1, inplace = True)
print(eval.shape)
eval.head()
eval.to_csv("eval.csv")

(1000, 13)


In [3]:
# Get train.csv
train = dfmanual.iloc[1000:]
train.drop(['label2'], axis=1, inplace = True)
print(train.shape)
train.head()

(2000, 13)


,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url,Sentiment Analysis (Label)
1000,Thu Oct 14 23:58:00 +0000 2021,1.448800e+18,Thinking about a conspiraboomer paying some CO...,1.0,0.0,1.0,5.0,en,38997531.0,1.448800286349435e+18,8000Gnat,https://twitter.com/8000Gnat/status/1448800286...,Negative
1001,Thu Oct 14 23:57:43 +0000 2021,1.448800e+18,@aja9696 @civillibertari2 @POTUS That's litera...,0.0,1.0,0.0,1.0,en,1.352370530813264e+18,1.4486984058999685e+18,trav088,https://twitter.com/trav088/status/14488002151...,Positive
1002,Thu Oct 14 23:57:42 +0000 2021,1.448800e+18,@POTUS I have a question Mr Presidente If I g...,0.0,0.0,0.0,0.0,en,1.4218579848508457e+18,1.4487674424870295e+18,Team_Litecoin,https://twitter.com/Team_Litecoin/status/14488...,Negative
1003,Thu Oct 14 23:57:37 +0000 2021,1.448800e+18,"@ziggystar18 @corybernardi Ironically, yes, be...",0.0,1.0,0.0,0.0,en,9.589402895314616e+17,1.4478417032645875e+18,Tupac_Katari123,https://twitter.com/Tupac_Katari123/status/144...,Positive
1004,Thu Oct 14 23:57:19 +0000 2021,1.448800e+18,@dysclinic And many become worse after covid v...,0.0,1.0,0.0,4.0,en,3032352520.0,1.4487448156555428e+18,ellecalendula,https://twitter.com/ellecalendula/status/14488...,Negative


## Finetuning Huggingface Model with 2k Train

In [4]:
# Transformers installation
! pip install transformers datasets
! pip3 install emoji
! pip install transformers[torch]
! pip install accelerate -U
! pip install evaluate

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.w

In [5]:
# Prepare dataset for finetuning on Colab
# Mappings are obtained from huggingface documentation
# https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis/blob/main/config.json
mappings = {
    'Negative': 0,
    'Positive': 2,
    'Neutral': 1
}
train['label'] = train['Sentiment Analysis (Label)'].map(mappings)
print(train[['full_text','label']].groupby('label').count())

       full_text
label           
0            413
1           1068
2            519


In [6]:
# Train test split
X=list(train['full_text'])
y=list(train['label'])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
print("Train test split done")

# Tokenize
model_name="finiteautomata/bertweet-base-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainencodings = tokenizer(X_train,truncation=True,padding=True)
testencodings = tokenizer(X_test,truncation=True,padding=True)
print("Tokenising done")

# Convert to datasets
class ADataset(Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels
  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels']=torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

trainds = ADataset(trainencodings,y_train)
testds = ADataset(testencodings,y_test)
print("Converted to datasets")

Train test split done


tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Tokenising done
Converted to datasets


In [7]:
# Training hyperparams
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=10
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainds,
    eval_dataset=testds,
    compute_metrics=compute_metrics
)
trainer.train()

Step,Training Loss
10,0.969700
20,0.866100
30,0.883800
40,0.752700
50,0.830100
60,0.874600
70,0.798900
80,0.672200
90,0.603200
100,0.794500


TrainOutput(global_step=800, training_loss=0.20691948636667803, metrics={'train_runtime': 127.9306, 'train_samples_per_second': 100.054, 'train_steps_per_second': 6.253, 'total_flos': 841962936729600.0, 'train_loss': 0.20691948636667803, 'epoch': 8.0})

In [8]:
trainer.save_model("./models")
!zip -r ./models.zip ./models

  adding: models/ (stored 0%)
  adding: models/model.safetensors (deflated 9%)
  adding: models/training_args.bin (deflated 52%)
  adding: models/config.json (deflated 51%)


## Evaluating Finetuned Huggingface Model on 1k Testset

In [2]:
test = df.iloc[0:1000]
print(test.head())
print(test.shape)
print("Dataframe imported and loaded")

                       created_at        id_str  \
0  Mon Feb 01 23:59:59 +0000 2021  1.356392e+18   
1  Mon Feb 01 23:59:58 +0000 2021  1.356392e+18   
2  Mon Feb 01 23:59:52 +0000 2021  1.356392e+18   
3  Mon Feb 01 23:59:38 +0000 2021  1.356392e+18   
4  Mon Feb 01 23:59:33 +0000 2021  1.356392e+18   

                                           full_text  quote_count  \
0  Here is today’s Idaho COVID-19 vaccine data at...          0.0   
1  Health care worker dies after second dose of C...          0.0   
2  Boris Johnson: Emergency Services to receive C...          0.0   
3  Think about it.... The same Liberal government...          0.0   
4  South Africa receives its first anti-Covid vac...          0.0   

   reply_count retweet_count  favorite_count lang             user_id_str  \
0          1.0           0.0             0.0   en            2284308170.0   
1          0.0           0.0             0.0   en              77677956.0   
2          0.0           0.0             0.0   

In [4]:
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("./models/")
classifier = pipeline('text-classification',model=model,tokenizer=tokenizer)

mappings = {
    'NEG': 'Negative',
    'POS': 'Positive',
    'NEU': 'Neutral'
}

def extract_label(full_text):
    try:
        label = mappings[classifier(full_text)[0]['label']]
        return label
    except:
        return 'Neutral'

label = []
for i in range(1000):
    if i%200 == 0:
        print("reached",i)
    label.append(extract_label(test['full_text'][i]))
test['label2'] = label

pd.crosstab(test['Sentiment Analysis (Label)'], test['label2'])

reached 0
reached 200
reached 400


Token indices sequence length is longer than the specified maximum sequence length for this model (284 > 128). Running this sequence through the model will result in indexing errors


reached 600
reached 800


label2,Negative,Neutral,Positive
Sentiment Analysis (Label),,,
Negative,228,11,11
Neutral,12,461,13
Positive,3,23,238


## Labelling Train Set

In [9]:
train2 = df.iloc[3000:8000]
train2.drop(['label2'], axis=1, inplace=True)
print(train2.head())
print(train2.shape)

                          created_at        id_str  \
3000  Tue May 25 23:59:43 +0000 2021  1.397342e+18   
3001  Tue May 25 23:59:29 +0000 2021  1.397342e+18   
3002  Tue May 25 23:59:26 +0000 2021  1.397342e+18   
3003  Tue May 25 23:59:16 +0000 2021  1.397342e+18   
3004  Tue May 25 23:58:54 +0000 2021  1.397341e+18   

                                              full_text  quote_count  \
3000  @MarioFreak6 @comradewatson @KaitMarieox I may...          0.0   
3001  According to a survey conducted by Fisher Phil...          2.0   
3002  @AlexBerenson Possibly, last March or April, t...          0.0   
3003  yes i got kick out facebook for covid and lock...          0.0   
3004  @meiselasb Covid cases are down, vaccines are ...          0.0   

      reply_count retweet_count  favorite_count lang          user_id_str  \
3000          0.0             0             0.0   en  1165971504212955136   
3001          0.0             2             3.0   en             91144092   
3002       

In [10]:
import time

tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("./models/")
classifier = pipeline('text-classification',model=model,tokenizer=tokenizer)

mappings = {
    'NEG': 'Negative',
    'POS': 'Positive',
    'NEU': 'Neutral'
}

def extract_label(full_text):
    try:
        label = mappings[classifier(full_text)[0]['label']]
        return label
    except:
        return 'Neutral'

label = []
start_time = time.time()
for i in range(3000,8000):
    if i%500 == 0:
        print("reached",i)
    label.append(extract_label(train2['full_text'][i]))
train2['Sentiment Analysis (Label)'] = label
train2['Sentiment Analysis (Label)'].value_counts()
print("time taken to label 5000 tweets (s):", time.time() - start_time)

reached 3000
reached 3500


Token indices sequence length is longer than the specified maximum sequence length for this model (315 > 128). Running this sequence through the model will result in indexing errors


reached 4000
reached 4500
reached 5000
reached 5500
reached 6000
reached 6500
reached 7000
reached 7500
time taken to label 5000 tweets (s): 249.18182706832886


In [11]:
train.drop(['label'], axis=1, inplace=True)
traindf = pd.concat([train,train2])
print(traindf.shape)
traindf.head()

(7000, 13)


,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url,Sentiment Analysis (Label)
1000,Thu Oct 14 23:58:00 +0000 2021,1.448800e+18,Thinking about a conspiraboomer paying some CO...,1.0,0.0,1.0,5.0,en,38997531.0,1.448800286349435e+18,8000Gnat,https://twitter.com/8000Gnat/status/1448800286...,Negative
1001,Thu Oct 14 23:57:43 +0000 2021,1.448800e+18,@aja9696 @civillibertari2 @POTUS That's litera...,0.0,1.0,0.0,1.0,en,1.352370530813264e+18,1.4486984058999685e+18,trav088,https://twitter.com/trav088/status/14488002151...,Positive
1002,Thu Oct 14 23:57:42 +0000 2021,1.448800e+18,@POTUS I have a question Mr Presidente If I g...,0.0,0.0,0.0,0.0,en,1.4218579848508457e+18,1.4487674424870295e+18,Team_Litecoin,https://twitter.com/Team_Litecoin/status/14488...,Negative
1003,Thu Oct 14 23:57:37 +0000 2021,1.448800e+18,"@ziggystar18 @corybernardi Ironically, yes, be...",0.0,1.0,0.0,0.0,en,9.589402895314616e+17,1.4478417032645875e+18,Tupac_Katari123,https://twitter.com/Tupac_Katari123/status/144...,Positive
1004,Thu Oct 14 23:57:19 +0000 2021,1.448800e+18,@dysclinic And many become worse after covid v...,0.0,1.0,0.0,4.0,en,3032352520.0,1.4487448156555428e+18,ellecalendula,https://twitter.com/ellecalendula/status/14488...,Negative


In [12]:
traindf.to_csv('train.csv', index=False)